<a href="https://colab.research.google.com/github/andrelucasbaldaia/Imersao-IA/blob/main/Imers%C3%A3o_IA_Primeiro_projeto_utilizando_o_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalação do Gemini

In [25]:
!pip install -q -U google-generativeai

Importação das bibliotecas e da API Key

In [43]:
import numpy as np
import pandas as pd
import google.generativeai as genai
from google.colab import userdata
api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

Listar os modelos disponíveis

In [ ]:
"""
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
      print(m.name)
"""

Listar as versões disponíveis de Embeddings

In [51]:

for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)


models/embedding-001
models/text-embedding-004


Variáveis criadas para configurar o Gemini

In [ ]:
"""
generation_config = {
    "candidate_count": 1,
    "temperature": 0.5,
}

safety_settings = {
    "Harassment": "BLOCK_NONE",
    "Hate": "BLOCK_NONE",
    "Sexually Explicit": "BLOCK_NONE",
    "Dangerous Content":"BLOCK_NONE",
}
"""

Inicializando o modelo

In [34]:
model = genai.GenerativeModel(model_name="gemini-1.0-pro")

Criando um chat utilizando o Gemini

In [ ]:
"""
chat = model.start_chat(history=[])

prompt = input("Digite algo: ")

while prompt != "fim":
  response = chat.send_message(prompt)
  print("Resposta: ", response.text, "\n")
  prompt = input("Digite algo: ")
"""

In [ ]:
#Exemplo de embedding

title = "A próxima geração de IA para desenvolvedores"
sample_text = ("Título: A próxima geração de IA para desenvolvedores"
      "\n"
      "Artigo completo:\n"
      "\n"
      "Gemini API & Google AI Studio: Uma maneira acessível de explorar e criar protótipos com aplicações de IA generativa")

embeddings = genai.embed_content(model="models/embedding-001",
                                 content=sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")

print(embeddings)

In [54]:
DOCUMENT1 = {
  "título": "Operando o Sistema de Controle de Clima",
  "conteúdo": "O seu Googlecar possui um sistema de controle de clima que permite ajustar a temperatura e o fluxo de ar dentro do carro. Para operar o sistema de controle de clima, use os botões e botões localizados no console central.  Temperatura: O botão de temperatura controla a temperatura dentro do carro. Gire o botão no sentido horário para aumentar a temperatura ou no sentido anti-horário para diminuir a temperatura. Fluxo de ar: O botão de fluxo de ar controla a quantidade de ar circulando dentro do carro. Gire o botão no sentido horário para aumentar o fluxo de ar ou no sentido anti-horário para diminuir o fluxo de ar. Velocidade do ventilador: O botão de velocidade do ventilador controla a velocidade do ventilador. Gire o botão no sentido horário para aumentar a velocidade do ventilador ou no sentido anti-horário para diminuir a velocidade do ventilador. Modo: O botão de modo permite selecionar o modo desejado. Os modos disponíveis são: Automático: O carro ajustará automaticamente a temperatura e o fluxo de ar para manter um nível confortável. Refrigerar: O carro soprará ar frio para dentro do carro. Aquecer: O carro soprará ar quente para dentro do carro. Descongelar: O carro soprará ar quente no para-brisa para desembaçá-lo."
}

DOCUMENT2 = {
  "título": "Tela sensível ao toque",
  "conteúdo": "O seu Googlecar possui uma grande tela sensível ao toque que fornece acesso a diversos recursos, incluindo navegação, entretenimento e controle de clima. Para usar a tela sensível ao toque, basta tocar no ícone desejado. Por exemplo, você pode tocar no ícone \"Navegação\" para obter instruções para o seu destino ou tocar no ícone \"Música\" para tocar suas músicas favoritas."
}

DOCUMENT3 = {
  "título": "Troca de marchas",
  "conteúdo": "O seu Googlecar possui transmissão automática. Para trocar de marcha, basta mover a alavanca seletora para a posição desejada. Park: Esta posição é usada quando você está estacionado. As rodas estão travadas e o carro não pode se mover. Ré: Esta posição é usada para dar ré. Neutro: Esta posição é usada quando você está parado em um semáforo ou no trânsito. O carro não está engatado e não se moverá a menos que você pressione o pedal do acelerador. Drive: Esta posição é usada para dirigir para frente. Low: Esta posição é usada para dirigir na neve ou em outras condições escorregadias."
}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]


In [57]:
df = pd.DataFrame(documents)
df


,título,conteúdo
0,Operando o Sistema de Controle de Clima,O seu Googlecar possui um sistema de controle ...
1,Tela sensível ao toque,O seu Googlecar possui uma grande tela sensíve...
2,Troca de marchas,O seu Googlecar possui transmissão automática....


In [58]:
model = "models/embedding-001"

In [59]:
def embed_fn(title, text):
  return float(genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"])

In [68]:
df["embeddings"] = df.apply(lambda row: embed_fn(row["título"], row["conteúdo"]), axis=1)
df

,título,conteúdo,embeddings
0,Operando o Sistema de Controle de Clima,O seu Googlecar possui um sistema de controle ...,"[-0.019291598, -0.022275448, -0.024370149, 0.0..."
1,Tela sensível ao toque,O seu Googlecar possui uma grande tela sensíve...,"[0.023562351, -0.035511028, 0.023264417, 0.008..."
2,Troca de marchas,O seu Googlecar possui transmissão automática....,"[-0.0051401346, -0.015164986, -0.005466612, 0...."


In [69]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")
  produtos_escalares = np.dot(np.stack(df["embeddings"]), list(embedding_da_consulta.values())[0])
  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["conteúdo"]

In [76]:
consulta = "Como faço para trocar marchas em um carro do google?"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

O seu Googlecar possui transmissão automática. Para trocar de marcha, basta mover a alavanca seletora para a posição desejada. Park: Esta posição é usada quando você está estacionado. As rodas estão travadas e o carro não pode se mover. Ré: Esta posição é usada para dar ré. Neutro: Esta posição é usada quando você está parado em um semáforo ou no trânsito. O carro não está engatado e não se moverá a menos que você pressione o pedal do acelerador. Drive: Esta posição é usada para dirigir para frente. Low: Esta posição é usada para dirigir na neve ou em outras condições escorregadias.


In [79]:
prompt = f"Reecreva esse texto de uma forma mais descontraída, sem adicionar informações que não façam parte do texto original: {trecho}"

model_2 = genai.GenerativeModel(model_name="gemini-1.0-pro")
response = model_2.generate_content(prompt)
print(response.text)

**Guia estiloso para o câmbio do seu Googlecar:**

Tá ligado no seu Googlecar? Ele tem uma transmissão automática, tipo um parça que faz as trocas de marcha na moleza. É só você movimentar o "seletor de marcha" pra onde quiser:

* **Park:** É o modo "relax", quando você tá estacionadão. As rodinhas ficam quietinhas, tipo um bebê dormindo.
* **Ré:** É o modo "bora voltar". Pra quando você quer dar aquela ré especial.
* **Neutro:** É o modo "espera aí". Pra quando você tá parado no farol ou no trânsito. O carro fica de boa, só esperando você pisar no acelerador.
* **Drive:** É o modo "vamo nessa". Pra quando você quer rodar pra frente, curtindo a paisagem.
* **Low:** É o modo "cautela no gelo". Pra quando a coisa ficar meio escorregadia, tipo neve ou sabão no chão.
